# Лабораторная работа №4: Случайный лес

1. Бейзлайн sklearn
2. Улучшение через подбор гиперпараметров
3. Собственная имплементация

## Импорт библиотек
Импортируем библиотеки для работы с Random Forest и метриками качества.

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import openml
import kagglehub
import os
import warnings
warnings.filterwarnings('ignore')
print("Импорт завершён")

Импорт завершён


### Датасет классификации: APS Failure at Scania Trucks
Загружаем данные, заполняем пропуски медианой, сэмплируем и разделяем на train/test со стратификацией.

In [13]:
# Загрузка классификации
dataset = openml.datasets.get_dataset(41138)
X_clf, y_clf, _, _ = dataset.get_data(target=dataset.default_target_attribute)
y_clf_enc = (y_clf == 'pos').astype(int)
imputer = SimpleImputer(strategy='median')
X_clf_imp = pd.DataFrame(imputer.fit_transform(X_clf), columns=X_clf.columns)
np.random.seed(42)
idx = np.random.choice(len(X_clf_imp), 15000, replace=False)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf_imp.iloc[idx], y_clf_enc.iloc[idx], test_size=0.2, random_state=42, stratify=y_clf_enc.iloc[idx]
)
print(f"Классификация: {X_clf_train.shape}")

Классификация: (12000, 170)


### Датасет регрессии: Avocado Prices
Загружаем данные о ценах авокадо. Кодируем категориальные признаки и разделяем на train/test.

In [14]:
# Загрузка регрессии: Avocado Prices
path = kagglehub.dataset_download("neuromusic/avocado-prices")
df = pd.read_csv(os.path.join(path, "avocado.csv"))
df['type_enc'] = LabelEncoder().fit_transform(df['type'])
df['region_enc'] = LabelEncoder().fit_transform(df['region'])
features = ['Total Volume', '4046', '4225', '4770', 'Total Bags', 'year', 'type_enc', 'region_enc']
X_reg = df[features].values
y_reg = df['AveragePrice'].values
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
print(f"Регрессия: {X_reg_train.shape}")

Регрессия: (14599, 8)


## 2. Бейзлайн

Обучаем базовый RandomForestClassifier с 50 деревьями. Random Forest — ансамблевый метод, комбинирующий множество деревьев решений для повышения качества и устойчивости.

In [15]:
# Бейзлайн классификация
rf_clf_base = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
rf_clf_base.fit(X_clf_train, y_clf_train)
y_pred_base = rf_clf_base.predict(X_clf_test)
y_proba_base = rf_clf_base.predict_proba(X_clf_test)[:, 1]
f1_base = f1_score(y_clf_test, y_pred_base)
roc_base = roc_auc_score(y_clf_test, y_proba_base)
print(f"=== БЕЙЗЛАЙН: Random Forest (n=50) ===")
print(f"F1: {f1_base:.4f}, ROC-AUC: {roc_base:.4f}")

=== БЕЙЗЛАЙН: Random Forest (n=50) ===
F1: 0.7170, ROC-AUC: 0.9762


Обучаем базовый Random Forest для регрессии с 50 деревьями. Случайный лес устойчив к переобучению благодаря усреднению предсказаний множества деревьев.

In [16]:
# Бейзлайн регрессия
rf_reg_base = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
rf_reg_base.fit(X_reg_train, y_reg_train)
y_pred_reg_base = rf_reg_base.predict(X_reg_test)
rmse_base = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_base))
r2_base = r2_score(y_reg_test, y_pred_reg_base)
print(f"=== БЕЙЗЛАЙН: Random Forest Regressor ===")
print(f"RMSE: {rmse_base:.4f}, R²: {r2_base:.4f}")

=== БЕЙЗЛАЙН: Random Forest Regressor ===
RMSE: 0.1630, R²: 0.8346


## 3. Улучшение бейзлайна

Подбор гиперпараметров Random Forest методом GridSearchCV: число деревьев `n_estimators`, глубина `max_depth` и минимальный размер листа `min_samples_leaf`. Оптимизируем по F1-score.

In [17]:
# GridSearch классификация
param_grid = {'n_estimators': [50, 100], 'max_depth': [10, 15, 20], 'min_samples_leaf': [1, 2]}
grid_clf = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_clf.fit(X_clf_train, y_clf_train)
print(f"Лучшие параметры: {grid_clf.best_params_}")
y_pred_imp = grid_clf.predict(X_clf_test)
y_proba_imp = grid_clf.predict_proba(X_clf_test)[:, 1]
f1_imp = f1_score(y_clf_test, y_pred_imp)
roc_imp = roc_auc_score(y_clf_test, y_proba_imp)
print(f"=== УЛУЧШЕННЫЙ ===")
print(f"F1: {f1_imp:.4f} ({f1_imp-f1_base:+.4f})")
print(f"ROC-AUC: {roc_imp:.4f}")

Лучшие параметры: {'max_depth': 20, 'min_samples_leaf': 2, 'n_estimators': 50}
=== УЛУЧШЕННЫЙ ===
F1: 0.7455 (+0.0285)
ROC-AUC: 0.9839


Подбор гиперпараметров для регрессии: число деревьев, глубина и минимальный размер листа. Больше деревьев обычно улучшает качество, но увеличивает время обучения.

In [18]:
# GridSearch регрессия
grid_reg = GridSearchCV(RandomForestRegressor(random_state=42, n_jobs=-1), 
                        {'n_estimators': [50, 100], 'max_depth': [10, 15], 'min_samples_leaf': [1, 5]}, 
                        cv=3, scoring='r2', n_jobs=-1)
grid_reg.fit(X_reg_train, y_reg_train)
print(f"Лучшие параметры: {grid_reg.best_params_}")
y_pred_reg_imp = grid_reg.predict(X_reg_test)
rmse_imp = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_imp))
r2_imp = r2_score(y_reg_test, y_pred_reg_imp)
print(f"=== УЛУЧШЕННЫЙ ===")
print(f"RMSE: {rmse_imp:.4f} ({rmse_imp-rmse_base:+.4f})")
print(f"R²: {r2_imp:.4f} ({r2_imp-r2_base:+.4f})")

/Users/ivan/dev/ml-7th-sem-labs/.venv/lib/python3.14/site-packages/sklearn/utils/parallel.py:144: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/Users/ivan/dev/ml-7th-sem-labs/.venv/lib/python3.14/site-packages/sklearn/utils/parallel.py:144: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/Users/ivan/dev/ml-7th-sem-labs/.venv/lib/python3.14/site-packages/sklearn/utils/parallel.py:144: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/Users/ivan/dev/ml-7th-sem-l

Лучшие параметры: {'max_depth': 15, 'min_samples_leaf': 1, 'n_estimators': 100}
=== УЛУЧШЕННЫЙ ===
RMSE: 0.1653 (+0.0023)
R²: 0.8299 (-0.0047)


## 4. Собственная имплементация

Реализуем алгоритм Random Forest с нуля:
- **Бэггинг**: каждое дерево обучается на bootstrap-выборке (с возвращением)
- **Случайные подпространства**: для каждого дерева выбираем sqrt(n_features) признаков
- **Голосование**: усреднение предсказаний (регрессия) или вероятностей (классификация)

In [19]:
class MyRandomForest:
    """Случайный лес"""
    def __init__(self, n_estimators=10, max_depth=10, max_features='sqrt', task='classification', random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.task = task
        self.random_state = random_state
        self.trees = []
        self.feature_indices = []
    
    def _get_n_features(self, n_total):
        if self.max_features == 'sqrt':
            return int(np.sqrt(n_total))
        return n_total
    
    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        n_samples, n_features = X.shape
        n_select = self._get_n_features(n_features)
        np.random.seed(self.random_state)
        
        for i in range(self.n_estimators):
            idx = np.random.choice(n_samples, n_samples, replace=True)
            feat_idx = np.random.choice(n_features, n_select, replace=False)
            self.feature_indices.append(feat_idx)
            
            if self.task == 'classification':
                tree = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state + i)
            else:
                tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state + i)
            tree.fit(X[idx][:, feat_idx], y[idx])
            self.trees.append(tree)
        return self
    
    def predict_proba(self, X):
        X = np.array(X)
        probas = np.zeros((len(X), 2))
        for tree, feat_idx in zip(self.trees, self.feature_indices):
            probas += tree.predict_proba(X[:, feat_idx])
        return probas / len(self.trees)
    
    def predict(self, X):
        X = np.array(X)
        if self.task == 'classification':
            return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)
        else:
            preds = np.zeros(len(X))
            for tree, feat_idx in zip(self.trees, self.feature_indices):
                preds += tree.predict(X[:, feat_idx])
            return preds / len(self.trees)

print("MyRandomForest реализован!")

MyRandomForest реализован!


Тестируем собственную реализацию случайного леса на задаче классификации. Сравниваем с бейзлайном sklearn по метрикам F1 и ROC-AUC.

In [20]:
# Своя классификация
my_rf_clf = MyRandomForest(n_estimators=50, max_depth=10, task='classification')
my_rf_clf.fit(X_clf_train.values, y_clf_train.values)
my_pred = my_rf_clf.predict(X_clf_test.values)
my_proba = my_rf_clf.predict_proba(X_clf_test.values)[:, 1]
my_f1 = f1_score(y_clf_test, my_pred)
my_roc = roc_auc_score(y_clf_test, my_proba)
print(f"=== СВОЯ: Классификация ===")
print(f"F1: {my_f1:.4f} (sklearn: {f1_base:.4f})")
print(f"ROC-AUC: {my_roc:.4f}")

=== СВОЯ: Классификация ===
F1: 0.6122 (sklearn: 0.7170)
ROC-AUC: 0.9778


Тестируем собственную реализацию случайного леса на задаче регрессии. Усредняем предсказания деревьев для получения финального результата.

In [21]:
# Своя регрессия
my_rf_reg = MyRandomForest(n_estimators=50, max_depth=10, task='regression')
my_rf_reg.fit(X_reg_train, y_reg_train)
my_pred_reg = my_rf_reg.predict(X_reg_test)
my_rmse = np.sqrt(mean_squared_error(y_reg_test, my_pred_reg))
my_r2 = r2_score(y_reg_test, my_pred_reg)
print(f"=== СВОЯ: Регрессия ===")
print(f"RMSE: {my_rmse:.4f} (sklearn: {rmse_base:.4f})")
print(f"R²: {my_r2:.4f} (sklearn: {r2_base:.4f})")

=== СВОЯ: Регрессия ===
RMSE: 0.2702 (sklearn: 0.1630)
R²: 0.5456 (sklearn: 0.8346)


## Итоговая сводка

Сравниваем Random Forest: бейзлайн sklearn, улучшенный с подобранными гиперпараметрами и собственную реализацию на основе бэггинга деревьев.

In [22]:
print("="*70)
print("ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №4 (Random Forest)")
print("="*70)
print(f"\n{'КЛАССИФИКАЦИЯ':-^70}")
print(f"{'Модель':<30} {'F1':<12} {'ROC-AUC':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {f1_base:<12.4f} {roc_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {f1_imp:<12.4f} {roc_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_f1:<12.4f} {my_roc:<12.4f}")
print(f"\n{'РЕГРЕССИЯ (Avocado Prices)':-^70}")
print(f"{'Модель':<30} {'RMSE':<12} {'R²':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {rmse_base:<12.4f} {r2_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {rmse_imp:<12.4f} {r2_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_rmse:<12.4f} {my_r2:<12.4f}")

ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №4 (Random Forest)

----------------------------КЛАССИФИКАЦИЯ-----------------------------
Модель                         F1           ROC-AUC     
------------------------------------------------------
Бейзлайн sklearn               0.7170       0.9762      
Улучшенный sklearn             0.7455       0.9839      
Своя реализация                0.6122       0.9778      

----------------------РЕГРЕССИЯ (Avocado Prices)----------------------
Модель                         RMSE         R²          
------------------------------------------------------
Бейзлайн sklearn               0.1630       0.8346      
Улучшенный sklearn             0.1653       0.8299      
Своя реализация                0.2702       0.5456      
